In [1]:
import os
import numpy as np
import open3d as o3d
from open3d import JVisualizer
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from IPython.display import clear_output

In [2]:
os.chdir("/home/vglasov/Reseach/LU-Net-pytorch/")
import config

In [74]:
class Pcd2ImageTransform:
    def __init__(self, shape=config.shape, angular=config.angular):
        self.height, self.width = shape
        y_angular, x_angular = angular
        self.x_delta, self.y_delta = x_angular / self.width, y_angular / self.height
    
    def fit(self, pcd, label_list):
        if pcd.is_empty():
            raise RuntimeError("Point Cloud must be non-empty")
            
        self.pcd = pcd
        x, y, z = np.array(self.pcd.points).T
        r = np.sqrt(x**2 + y**2 + z**2)
        azimuth_angle = np.arctan2(x, y)
        elevation_angle = np.arcsin(z / r)
        
        x_img = np.floor(azimuth_angle / self.x_delta).astype(int)
        y_img = np.floor(elevation_angle / self.y_delta).astype(int)
        
        x_img -= x_img.min()
        y_img -= y_img.min()
        
        self.pcd_labels = np.zeros(len(self.pcd.points), dtype=int)
        for x_min, y_min, z_min, x_max, y_max, z_max in label_list:            
            self.pcd_labels[np.all([x_min <= x,
                                    y_min <= y, 
                                    z_min <= z,
                                    x <= x_max,
                                    y <= y_max,
                                    z <= z_max], axis=0)] = 1
        
        self.transformation = pd.DataFrame({'x': x, 
                                            'y': y,
                                            'z': z,
                                            'x_img': x_img,
                                            'y_img': y_img,
                                            'r': r,
                                            'label': self.pcd_labels})        
        return self
    
    def transform(self, pcd, label_list):
        X = self.transformation.groupby(['y_img', 'x_img'])[['r', 'label']].mean()
        range_image = X['r'].unstack().values[::-1, :]
        mask = np.isfinite(X['r'].unstack().values[::-1, :])
        labels = X['label'].unstack().values[::-1, :]
        
        return range_image, mask, np.nan_to_num(labels) > 0
    
    def fit_transform(self, pcd, label_list):
        self.fit(pcd, label_list)
        return self.transform(pcd, label_list)
    
    def inverse_transform(self, prediction):
        x_idx_pred, y_idx_pred = np.where(labels)
        prediction = np.zeros(len(self.transformation))
        for x, y in zip(*np.where(labels)):
            prediction[(self.transformation['x_img'] == x) & (self.transformation['y_img'] == y)] = 1
        pcd = o3d.PointCloud(self.pcd)
#         pcd.points = self.pcd
        pcd.colors = o3d.Vector3dVector(np.vstack((prediction,
                                                   np.zeros_like(prediction),
                                                   prediction)).T)
        return pcd

In [75]:
x = np.random.randint(0, 10, 4)
np.vstack((x, x))

array([[9, 0, 2, 5],
       [9, 0, 2, 5]])

In [76]:
%%time
pts_path = os.path.join(config.dataset, "LCAS_20160523_1200_1218_pcd", "1464001379.626087000.pcd")
labels_path = os.path.join(config.dataset, "LCAS_20160523_1200_1218_labels", "1464001379.626087000.txt")

fragment = o3d.read_point_cloud(pts_path)
labels_list = [list(map(float, f.split()[4:10])) for f in open(labels_path, "r").readlines()]


pcd2img = Pcd2ImageTransform().fit(fragment, labels_list)
X, mask, labels = pcd2img.transform(fragment, labels_list)

pcd2img.inverse_transform(labels)

CPU times: user 355 ms, sys: 0 ns, total: 355 ms
Wall time: 355 ms


geometry::PointCloud with 26456 points.

In [14]:
x_idx_pred, y_idx_pred = np.where(labels)

In [27]:
pcd2img = Pcd2ImageTransform().fit(fragment, labels_list)

In [30]:
len(pcd2img.transformation)

26456